--- Day 14: Extended Polymerization ---

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

NNCB

`CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C`

The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

    The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
    The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
    The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

Template:     `NNCB`

After step 1: `NCNBCHB`

After step 2: `NBCCNBBBCBHCB`

After step 3: `NBBBCNCCNBBNBNBBCHBHHBCHB`

After step 4: `NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB`

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?


# Attempt 1: Brute Force

The first attempt is to just compute the entire polymer directly. Since the number of elements increases exponentially this is clearly not a good solution for a large number of steps.

In [ ]:
from collections import Counter
import copy

class polymer:
    polymer = []
    frequencies = []
    letters = []
    rules = {}
    def __init__(self,p,rules):
        self.polymer = p
        self.rules = rules
        # compute frequencies
        self.letters = Counter(self.polymer).keys() 
        self.frequencies = Counter(self.polymer).values()
        
    def insert_rule(self):
        cp = copy.copy(self.polymer)
        cnt = 1
        for i in range(len(cp)-1):
            ins = self.rules[cp[i]+cp[i+1]]
            self.polymer.insert(cnt,ins)
            self.update_frequency()
            cnt += 2
            
    def update_frequency(self):
        self.letters = Counter(self.polymer).keys() 
        self.frequencies = Counter(self.polymer).values()

r = {
"CH": "B", "HH": "N", "CB": "H", "NH": "C", "HB": "C", "HC": "B", "HN": "C", "NN": "C",
"BH": "H", "NC": "B", "NB": "B", "BN": "B", "BB": "N", "BC": "B", "CC": "N", "CN": "C"
}
p = polymer(['N','N','C','B'],r)
print(p.polymer)
for i in range(10):
    p.insert_rule()
    print(p.letters)
    print(p.frequencies)

# Attempt 2: Linear Mapping

Instead we realise that we do not need to compute the actual polymer; we just need to know its frequencies.

We represent the polymer as a list of pairs, each pair has two children. To compute frequencies of each pair after one step we can create a linear map $M$. $N$ steps are then computed by applying $M^N$.

To compute the frequency of an element (as opposed to the frequency of a pair) we sum up over the left component of each pair and then add the last element of the input string.

In [ ]:
import numpy as np

class frequency_polymer:
    children = {}
    matrix = []
    initial = []
    last_entry = -1
    
    def __init__(self,polymer,rules):
        self.last_entry = polymer[-1]
        self.set_up_children(rules)
        self.set_up_initial_frequency(polymer)
        
        # set up matrix
        self.matrix = np.zeros((len(self.children),len(self.children)))
        for i,row in enumerate(self.children.keys()):
            for j,column in enumerate(self.children.keys()):
                ch1 = self.children[column][0]
                ch2 = self.children[column][1]               
                if ch1 == row:
                    self.matrix[i,j] += 1
                if ch2 == row:
                    self.matrix[i,j] += 1
            
    def set_up_children(self,rules):
        vector = list(rules.keys())
        for i in range(len(vector)):
            self.children[vector[i]] = [vector[i][0]+rules[vector[i]],
                                             rules[vector[i]]+vector[i][1]]
        
    def set_up_initial_frequency(self,polymer):
        self.initial = np.zeros(len(self.children))
        vector = list(self.children.keys())
        for i,row in enumerate(vector):
            for char1,char2 in zip(polymer, polymer[1:]):
                if (char1+char2) == row:
                    self.initial[i] += 1
        
    def apply_matrix(self, it):
        return np.linalg.matrix_power(self.matrix,it)@self.initial
    
    def frequency(self, v):
        letters = {}
        for e in self.children.keys():
            letters[e[0]] = 0
            letters[e[1]] = 0
        letters[self.last_entry] += 1
        for f,e in zip(v,self.children.keys()):
            letters[e[0]] += f
        return letters

r = {
    "CH": "B", "HH": "N", "CB": "H", "NH": "C", "HB": "C", "HC": "B", "HN": "C", "NN": "C",
    "BH": "H", "NC": "B", "NB": "B", "BN": "B", "BB": "N", "BC": "B", "CC": "N", "CN": "C"
}

p = frequency_polymer(['N','N','C','B'],r)
v = p.apply_matrix(10)
p.frequency(v)
print(p.frequency(v))

And then applied to the larger example:

In [ ]:
with open('input_day14') as file:
    initial = [d for d in file.readline().split(",")]
    
    # read rules
    rules = {}
    lines = filter(None, (line.rstrip() for line in file))
    for line in lines:
        rules[line[0]+line[1]] = line[6]
        
    p = frequency_polymer(list(initial[0])[:-1],rules)

v = p.apply_matrix(40)
print(p.frequency(v))

In [ ]:
sol = max(p.frequency(v).values()) - min(p.frequency(v).values())
print(sol)

### Eigendecomposition

Since we can now represent each step as one application of the matrix, we could also decompose the matrix:
$$A = S\Lambda S^{-1},$$
where $\Lambda$ is a diagonal matrix eigenvectors and $S$ are the eigenvectors.

Then
$$A^N = S\Lambda^N S^{-1}$$

We can store $\Lambda, S, S^{-1}$ to more efficiently compute each application or even to predict the behaviour for steps$\rightarrow\infty$.

In [ ]:
from scipy.linalg import eig, inv

class frequency_polymer:
    children = {}
    matrix = []
    initial = []
    last_entry = -1
    eigenvec = []
    S = []
    Sinv = []
    
    def __init__(self,polymer,rules):
        self.last_entry = polymer[-1]
        self.set_up_children(rules)
        self.set_up_initial_frequency(polymer)
        
        # set up matrix
        self.matrix = np.zeros((len(self.children),len(self.children)))
        for i,row in enumerate(self.children.keys()):
            for j,column in enumerate(self.children.keys()):
                ch1 = self.children[column][0]
                ch2 = self.children[column][1]               
                if ch1 == row:
                    self.matrix[i,j] += 1
                if ch2 == row:
                    self.matrix[i,j] += 1
        self.eigenval,self.S = eig(self.matrix)
        self.Sinv = inv(self.S)
            
    def set_up_children(self,rules):
        vector = list(rules.keys())
        for i in range(len(vector)):
            self.children[vector[i]] = [vector[i][0]+rules[vector[i]],
                                             rules[vector[i]]+vector[i][1]]
        
    def set_up_initial_frequency(self,polymer):
        self.initial = np.zeros(len(self.children))
        vector = list(self.children.keys())
        for i,row in enumerate(vector):
            for char1,char2 in zip(polymer, polymer[1:]):
                if (char1+char2) == row:
                    self.initial[i] += 1
        
    def apply_matrix(self, it):
        eigvalit = np.array([e**it for e in self.eigenval])
        return np.round(np.real(self.S@np.diag(eigvalit)@self.Sinv)@self.initial)
    
    def frequency(self, v):
        letters = {}
        for e in self.children.keys():
            letters[e[0]] = 0
            letters[e[1]] = 0
        letters[self.last_entry] += 1
        for f,e in zip(v,self.children.keys()):
            letters[e[0]] += f
        return letters

v = p.apply_matrix(40)
print(p.frequency(v))

Let's plot the percentage of the polymer made up by each element

In [ ]:
import matplotlib.pyplot as plt

n_iter = 30
vplot = []
total = []
for i in range(n_iter):
    v = p.apply_matrix(i)
    vplot.append(p.frequency(v))
    total.append(sum(vplot[i].values()))

for j in vplot[0].keys():
    vj = [vplot[i][j] for i in range(n_iter)]
    plt.plot(range(n_iter),np.array(vj)/np.array(total),label=j)
    plt.legend()